In [1]:
from timeit import default_timer as timer
import sys
import os
import numpy as np
import pandas as pd
import re
import string
import reverse_geocoder as rg
from iso3166 import countries
import pyarrow.parquet as pq
from glob import glob

In [2]:
# Local
print('Hostname:', socket.gethostname())
if  'samuel' in socket.gethostname().lower():
    path_to_data='../../data'
# Cluster
else:
    path_to_data='/scratch/spf248/twitter/data'

path_to_tweets=os.path.join(path_to_data,'decahose/parsed/tweets/tweets-with-geocoordinates-or-place-extract','*.parquet')
print(path_to_tweets)

NameError: name 'socket' is not defined

In [ ]:
print("Load...")
start = timer()

tweets=pq.ParquetDataset(glob(path_to_tweets)).read().to_pandas()
print('# Tweets:',tweets.shape[0])

print("Done in", round(timer()-start), "sec")

In [ ]:
print("Drop Missing Geocoordinates...")
start = timer()

tweets=tweets[(-tweets.tweet_longitude.isnull())&(-tweets.tweet_latitude.isnull())].copy()
print('# Tweets:',tweets.shape[0])

print("Done in", round(timer()-start), "sec")

In [ ]:
print("Reverse geocode representative point...")
start = timer()

tweets['tweet_point'] = list(zip(tweets['tweet_latitude'], tweets['tweet_longitude']))

reversed_geocodes = pd.DataFrame(rg.search(list(tweets['tweet_point'].values))).drop(['lat','lon'],1)

reversed_geocodes['country'] = reversed_geocodes['cc'].apply(lambda x:countries.get(x).name)

print("Done in", round(timer()-start), "sec")

In [ ]:
pd.concat([tweets.drop('tweet_point',1),reversed_geocodes],1)

In [ ]:
# tweets = pd.concat([tweets.drop('tweet_point',1),reversed_geocodes],1)

# del reversed_geocodes

In [56]:
tweets.head()

,tweet_id,created_at,text,tweet_lang,user_id,user_location,place_id,tweet_longitude,tweet_latitude,name,admin1,admin2,cc,country
0,812406347430031361,2016-12-23 21:15:41,Almuerzo navideño (@ Mar y Punto in Miraflores...,es,79900019,"Lima, Peru",005bf2190435cf68,-77.009212,-12.13052,Surco,Lima,Lima,PE,Peru
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guapimirim,Rio de Janeiro,Guapimirim,BR,Brazil
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Festus,Missouri,Jefferson County,US,United States of America
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sama,Asturias,Province of Asturias,ES,Spain
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,les Corts,Catalonia,Provincia de Barcelona,ES,Spain
